<a href="https://colab.research.google.com/github/ednadev/Data-Scientist/blob/master/06_Deep_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import pdb

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='../../data/', 
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/', 
                                           train=False,
                                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.ReLU())
    self.layer2=nn.MaxPool2d(kernel_size=2, stride=2)
    self.layer3=nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.ReLU())
    self.layer4=nn.MaxPool2d(kernel_size=2, stride=2)
    self.layer5=nn.Linear(7*7*32, num_classes)

  def forward(self, x):
    # 2.
    # pdb.set_trace()
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = out.reshape(out.size(0), -1) # 2차원을 1차원으로 펼치는 과정
    out = self.layer5(out)
    return out

In [ ]:
model = ConvNet(num_classes).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_step = len(train_loader) # 600번
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #1.
    # pdb.set_trace()

    preds = model(images)

    # 3.
    # pdb.set_trace()
    loss = loss_function(preds, labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if(i+1) % 100 ==0:
      print('Epoch [{}/{}], Step [{}/{}], Loss:{:.3f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
model.eval() # 모델을 eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images) # 위에 정의해놓은 forward() 가 호출된다.

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test Accuracy : {}%'.format(100 * correct / total))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
images.shape
images[0].size()
# 첫번째 이미지의 사이즈
images[0,0].size()

labels.size()

image = np.array(images[0, 0].detach().cpu()) # gpu에서 내린다.
label = labels[0].item()
# predicted = predicted[0].item()

plt.imshow(image*255)
print('label ', label)
print('pred', predicted)

In [ ]:
for idx in range(30):
  image = np.array(images[idx, 0].detach().cpu())
  label = labels[idx].item()
  pred = predicted[idx].item()
  plt.figure(figsize=(2,2))
  plt.title(f'Label : {label} Predict : {pred}')
  plt.imshow(image * 255)